In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
print(os.getcwd())
if os.getcwd() != '/content/drive/My Drive/Dissertation':
    os.chdir('/content/drive/My Drive/Dissertation')

Mounted at /content/drive
/content


In [ ]:
!pip install sacrebleu
!pip install sentencepiece
!pip install tqdm

     |████████████████████████████████| 71kB 6.0MB/s 
     |████████████████████████████████| 1.2MB 9.2MB/s 


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from math import sqrt
Tensor = torch.Tensor
from torch.autograd import Variable

from typing import Optional
import datetime
import time
import numpy as np
import pickle

import sentencepiece as spm
from tqdm import tqdm
from tqdm.notebook import tnrange
import sacrebleu

In [ ]:
# % cd sentencepiece/
# % cd build/
# ! cmake ..
# ! make -j $(nproc)
# ! sudo make install
# ! sudo ldconfig -v

In [ ]:
# % cd data/
# % cd data/es

In [ ]:
# ! unzip en-es.txt.zip

In [ ]:
# spm.__dict__

In [ ]:


# import subprocess
# # os.chdir('data')
# subprocess.run(
#             ["unzip", "en-es.txt.zip"],
#             stdout=subprocess.PIPE,
#             encoding="utf-8"
#         ).stdout.replace('‘', '\'').replace('’', '\'')
# os.chdir('..')
# taking first million

# def move(l, num):
#     inpfile = open(input_file+l, 'r', encoding="utf-8")
#     intfile = open(interim_file+l, 'w', encoding='utf-8')
#     with mosestokenizer.MosesPuntuationNormalizer(l) as normalizer:
#         for i in range(num):
#             intfile.write(normalizer(inpfile.readline()).lower())
#     inpfile.close()
#     intfile.close()
# move("fr", 10**6)
# move("en", 10**6)

In [ ]:
# input_file = interim_file
# vocab_size = 8000

# for l in ['fr', 'en']:
#     if not os.path.exists(f"{model_file}{l}.model"):
#         spm.SentencePieceTrainer.Train(
#             input=f"{input_file}{l}",
#             model_prefix=f"{model_file}{l}",
#             model_type='bpe',
#             vocab_size=vocab_size,
#             pad_id=3,
#             pad_piece='<p>',
#             bos_piece='<s>',
#             eos_piece='</s>'
#             )
    

In [ ]:
from math import sin, cos, sqrt
class MultiHeadAttention(nn.Module):
    def __init__(self, model_dim: int, heads: int, dropout: float = 0.1):
        super().__init__()

        if torch.cuda.is_available():
            dev = "cuda:0"
        else:
            dev = "cpu"

        device = torch.device(dev)

        self.model_dim: int = model_dim
        self.heads: int = heads
        self.d_k: int = model_dim // heads
        self.dropout: nn.Dropout = nn.Dropout(dropout).to(device)
        self.add_module('dropout', self.dropout)

        self.w_key: nn.Linear = nn.Linear(model_dim, model_dim).to(device)
        self.w_query: nn.Linear = nn.Linear(model_dim, model_dim).to(device)
        self.w_value: nn.Linear = nn.Linear(model_dim, model_dim).to(device)
        self.add_module('w_key', self.w_key)
        self.add_module('w_query', self.w_query)
        self.add_module('w_value', self.w_value)

        self.final_layer: nn.Linear = nn.Linear(model_dim, model_dim).to(device)
        self.add_module('final_layer', self.final_layer)

    def forward(self,
                q: Tensor,
                k: Tensor,
                v: Tensor,
                mask: Optional[Tensor] = None):

        # if encoder_output is not None: print("enc_output == True")
        # print(f'x.shape: {x.shape}')

        batch_size = q.size(0)
        queries = self.w_query(q).view(batch_size, -1, self.heads, self.d_k)
        keys = self.w_key(k).view(batch_size, -1, self.heads, self.d_k)
        values = self.w_value(v).view(batch_size, -1, self.heads, self.d_k)

        # we have batch_size, seq_len, heads, d_k tensor

        keys.transpose_(1, 2)
        queries.transpose_(1, 2)
        values.transpose_(1, 2)

        interim_result = torch.matmul(queries, keys.transpose(-2, -1)) / sqrt(self.d_k)

        if mask is not None:
            mask = mask.unsqueeze(1)
            interim_result = interim_result.masked_fill(mask == 0, -1e9)

        interim_result = F.softmax(interim_result, dim=-1)

        if self.dropout is not None:
            interim_result = self.dropout(interim_result)

        interim_result = torch.matmul(interim_result, values)

        concatenated_result = interim_result.transpose(1, 2).contiguous().view(batch_size, -1, self.model_dim)

        final_result = self.final_layer(concatenated_result)

        return final_result



class PositionalEncoding(nn.Module):

    """A parameter less module that concatenates a number of sine signals at the end of the embedded vectors."""

    def __init__(self, model_dim: int, max_length: int, dropout: float = 0.1):
        super().__init__()

        if torch.cuda.is_available():
            dev = "cuda:0"
        else:
            dev = "cpu"

        device = torch.device(dev)

        self.model_dim: int = model_dim
        self.dropout: nn.Dropout = nn.Dropout(dropout)
        self.add_module('dropout', self.dropout)

        position_vector: Tensor = torch.zeros(max_length, model_dim, requires_grad=False).to(device)
        # arange = torch.arange(max_length)

        # note to self: this appear to work right now.
        for pos in range(max_length):
            for i in range(0, model_dim, 2):
                # Follwing the formula provided in the paper.
                position_vector[pos, i] = sin(pos / (10000 ** ((2 * i) / model_dim)))
                position_vector[pos, i+1] = cos(pos / (10000 ** ((2 * (i + 1)) / model_dim)))

        # position_vector: max_seq_len x model_dim
        position_vector = position_vector.unsqueeze(0)

        # position_vector: 1 x max_seq_len x model_dim
        self.register_buffer('position_vector', position_vector)

    def forward(self, x: Tensor) -> Tensor:
        """
        The forward pass implementation of the Positional Embedding step.
        :param x: the tensor containing Batch x Seq_len x model_dim embeddings.
        :return: the embedded vector with some alterations.
        """

        x = x * sqrt(self.model_dim)
        sequence_length = x.size(1)
        x = x + Variable(self.position_vector[:, :sequence_length], requires_grad=False)
        x = self.dropout(x)
        return x


class FeedForward(nn.Module):

    """The utility module that is used to implement a feed-forward fully connected Neural Net for the
    encoder and decoder layers."""

    def __init__(self, model_dim: int, d_ff: int = 2048, dropout: float = 0.1):
        super().__init__()

        if torch.cuda.is_available():
            dev = "cuda:0"
        else:
            dev = "cpu"

        device = torch.device(dev)

        self.fc1: nn.Linear = nn.Linear(model_dim, d_ff).to(device)
        self.dropout: nn.Dropout = nn.Dropout(dropout).to(device)
        self.fc2: nn.Linear = nn.Linear(d_ff, model_dim).to(device)

        self.add_module('fc1', self.fc1)
        self.add_module('dropout', self.dropout)
        self.add_module('fc2', self.fc2)

    def forward(self, x: Tensor) -> Tensor:

        """
        Implements Feed-Forward algorithm with dropout.
        :param x: a tensor with last dim = model_dim
        :return: output from the NN
        """

        x = self.dropout(F.relu(self.fc1(x)))
        x = self.fc2(x)

        return x




class TransformerEncoderLayer(nn.Module):
    def __init__(self, model_dim: int, heads: int, d_ff: int,
                 dropout: Optional[float] = 0.1, norm_before: bool = False):
        super().__init__()

        if torch.cuda.is_available():
            dev = "cuda:0"
        else:
            dev = "cpu"

        device = torch.device(dev)

        self.model_dim: int = model_dim
        self.heads: int = heads
        self.dropout: float = dropout
        self.norm_before: bool = norm_before

        self.self_attn: MultiHeadAttention = MultiHeadAttention(model_dim,
                                                                heads,
                                                                dropout).to(device)
        self.add_module('self_attn', self.self_attn)

        self.attn_dropout: nn.Dropout = nn.Dropout(dropout).to(device)
        self.ffn_dropout: nn.Dropout = nn.Dropout(dropout).to(device)
        self.add_module('attn_dropout', self.attn_dropout)
        self.add_module('ffn_dropout', self.ffn_dropout)

        self.ffn: FeedForward = FeedForward(model_dim, d_ff, dropout).to(device)
        self.add_module('ffn', self.ffn)

        self.attn_norm: nn.LayerNorm = nn.LayerNorm(model_dim).to(device)
        self.ffn_norm: nn.LayerNorm = nn.LayerNorm(model_dim).to(device)
        self.add_module('attn_norm', self.attn_norm)
        self.add_module('ffn_norm', self.ffn_norm)

    def forward(self, x: Tensor, mask: Tensor = None) -> Tensor:
        y = x
        if self.norm_before:
            y = self.attn_norm(x)
        # print('encoder_attention')
        y = self.self_attn(y, y, y, mask)
        y = self.attn_dropout(y)
        x = x + y
        if not self.norm_before:
            x = self.attn_norm(x)

        y = x
        if self.norm_before:
            y = self.ffn_norm(x)
        y = self.ffn(y)
        y = self.ffn_dropout(y)
        x = x + y
        if not self.norm_before:
            x = self.ffn_norm(x)

        return x


class TransformerEncoder(nn.Module):
    def __init__(self, model_dim: int, d_ff: int, heads: int, num_blocks: int,
                 vocab_size: int, max_seq_len: Optional[int] = 80,
                 dropout: Optional[float] = 0.1, norm_before: bool = False):
        super().__init__()

        if torch.cuda.is_available():
            dev = "cuda:0"
        else:
            dev = "cpu"

        device = torch.device(dev)

        self.model_dim: int = model_dim
        self.heads: int = heads
        self.num_blocks: int = num_blocks
        self.vocab_size: int = vocab_size
        self.max_seq_len: int = max_seq_len

        self.word_embeddings: nn.Embedding = nn.Embedding(vocab_size, model_dim).to(device)
        self.positional_encoding: PositionalEncoding = PositionalEncoding(model_dim, max_seq_len).to(device)
        self.add_module('word_embeddings', self.word_embeddings)

        args = (model_dim, heads, d_ff, dropout, norm_before)
        # encoding_layer: TransformerEncoderLayer = TransformerEncoderLayer(*args).to(device)
        self.encoding_layers: List[TransformerEncoderLayer] = [TransformerEncoderLayer(*args).to(device)
                                                               for _ in range(num_blocks)]
        for i, layer in enumerate(self.encoding_layers):
            self.add_module(f'layer_{i}', layer)

    def forward(self, _input: Tensor, mask: Tensor):
        x = self.word_embeddings(_input)
        x = self.positional_encoding(x)
        # print(f"x.shape {x.shape}")
        for layer in self.encoding_layers:
            x = layer(x, mask)

        return x


class TransformerDecoderLayer(nn.Module):
    def __init__(self, model_dim: int, heads: int, d_ff: int,
                 dropout: Optional[float] = 0.1, norm_before: bool = False):
        super().__init__()

        if torch.cuda.is_available():
            dev = "cuda:0"
        else:
            dev = "cpu"

        device = torch.device(dev)

        self.model_dim: int = model_dim
        self.heads: int = heads
        self.d_ff: int = d_ff
        self.dropout: float = dropout
        self.norm_before: bool = norm_before

        self.self_attn_norm: nn.LayerNorm = nn.LayerNorm(model_dim).to(device)
        self.enc_dec_norm: nn.LayerNorm = nn.LayerNorm(model_dim).to(device)
        self.ffn_norm: nn.LayerNorm = nn.LayerNorm(model_dim).to(device)
        self.add_module('self_attn_norm', self.self_attn_norm)
        self.add_module('enc_dec_norm', self.enc_dec_norm)
        self.add_module('ffn_norm', self.ffn_norm)

        self.self_attn_dropout: nn.Dropout = nn.Dropout(dropout).to(device)
        self.enc_dec_dropout: nn.Dropout = nn.Dropout(dropout).to(device)
        self.ffn_dropout: nn.Dropout = nn.Dropout(dropout).to(device)
        self.add_module('self_attn_dropout', self.self_attn_dropout)
        self.add_module('enc_dec_dropout', self.self_attn_dropout)
        self.add_module('ffn_dropout', self.ffn_dropout)

        self.self_attn: MultiHeadAttention = MultiHeadAttention(model_dim, heads, dropout).to(device)
        self.enc_dec_attn: MultiHeadAttention = MultiHeadAttention(model_dim, heads, dropout).to(device)
        self.ffn: FeedForward = FeedForward(model_dim, d_ff).to(device)
        self.add_module('self_attn', self.self_attn)
        self.add_module('enc_dec_attn', self.enc_dec_attn)
        self.add_module('ffn', self.ffn)

    def forward(self, x: Tensor, encoder_output: Tensor, src_mask: Tensor, trg_mask: Tensor) -> Tensor:

        y = x
        if self.norm_before:
            y = self.self_attn_norm(x)
        # print('decoder_attention')
        y = self.self_attn(y, y, y, trg_mask)
        y = self.self_attn_dropout(y)
        x = x + y
        if not self.norm_before:
            x = self.self_attn_norm(x)

        y = x
        if self.norm_before:
            y = self.enc_dec_norm(x)
        # print('enc_dec_attn')
        y = self.enc_dec_attn(y, encoder_output, encoder_output,
                              src_mask)
        y = self.enc_dec_dropout(y)
        x = x + y
        if not self.norm_before:
            x = self.enc_dec_norm(x)

        y = x
        if self.norm_before:
            y = self.ffn_norm(x)
        y = self.ffn(y)
        y = self.ffn_dropout(y)
        x = x + y
        if not self.norm_before:
            x = self.ffn_norm(x)

        return x


class TransformerDecoder(nn.Module):
    def __init__(self, model_dim: int, d_ff: int, heads: int, num_blocks: int,
                 vocab_size: int, max_seq_len: Optional[int] = 80,
                 dropout: Optional[float] = 0.1, norm_before: bool = True):
        super().__init__()

        if torch.cuda.is_available():
            dev = "cuda:0"
        else:
            dev = "cpu"

        device = torch.device(dev)

        self.model_dim: int = model_dim
        self.heads: int = heads
        self.num_blocks: int = num_blocks
        self.vocab_size: int = vocab_size
        self.max_seq_len: int = max_seq_len
        self.dropout: float = dropout
        self.norm_before: bool = norm_before

        self.word_embeddings: nn.Embedding = nn.Embedding(vocab_size, model_dim).to(device)
        self.positional_embeddings: PositionalEncoding = PositionalEncoding(model_dim, max_seq_len).to(device)

        args = (model_dim, heads, d_ff, dropout, norm_before)

        # decoding_layer: TransformerDecoderLayer = TransformerDecoderLayer(*args).to(device)
        self.decoding_layers: List[TransformerDecoderLayer] = [TransformerDecoderLayer(*args).to(device)
                                                               for _ in range(num_blocks)]
        for i, layer in enumerate(self.decoding_layers):
            self.add_module(f'layer_{i}', layer)

    def forward(self, target: Tensor, encoder_output: Tensor, src_mask: Tensor, trg_mask: Tensor) -> Tensor:
        x: Tensor = self.word_embeddings(target)
        x = self.positional_embeddings(x)

        for layer in self.decoding_layers:
            x = layer(x, encoder_output, src_mask, trg_mask)

        return x


class Transformer(nn.Module):
    def __init__(self, src_vocab_size: int, trg_vocab_size: int,
                 model_dim: int, d_ff: int, heads: int, num_blocks: int,
                 max_seq_len: Optional[int] = 80, dropout: Optional[float] = 0.1,
                 norm_before: bool = False):
        super().__init__()

        if torch.cuda.is_available():
            dev = "cuda:0"
        else:
            dev = "cpu"

        device = torch.device(dev)

        self.encoder: TransformerEncoder = TransformerEncoder(model_dim, d_ff, heads, num_blocks,
                                                              src_vocab_size, max_seq_len, dropout,
                                                              norm_before).to(device)
        self.decoder: TransformerDecoder = TransformerDecoder(model_dim, d_ff, heads, num_blocks,
                                                              trg_vocab_size, max_seq_len, dropout,
                                                              norm_before).to(device)

        self.linear: nn.Linear = nn.Linear(model_dim, trg_vocab_size).to(device)

        self.add_module('encoder', self.encoder)
        self.add_module('decoder', self.decoder)
        self.add_module('linear', self.linear)

    def forward(self, src: Tensor, trg: Tensor, src_mask: Tensor, trg_mask: Tensor):
        enc_output = self.encoder(src, src_mask)
        dec_output = self.decoder(trg, enc_output, src_mask, trg_mask)
        output = self.linear(dec_output)
        del enc_output, dec_output
        torch.cuda.empty_cache()
        return output

    def save_model(self, file_name):
      torch.save(self.state_dict(), file_name)

from datetime import datetime
class Log:
    LOG, ERROR = 0, 1

    def __init__(self, outfile='data/.log', filename='data/logfile.log'):
        self.filename = filename
        self.outfile = outfile
        self.file_object = open(filename, 'a+', encoding='utf-8')
        self.line_num = 0
        print("LOGGING For seesion on: " + str(datetime.now()), file=self.file_object)

    def print(self, txt, type=LOG, shell=True):
        if shell: print(txt)
        prefix = "LOG ::" if type==Log.LOG else "ERROR ::"
        txt = f"{prefix} {str(datetime.now())} :: {txt}"
        print(txt, file=self.file_object)

    def close(self):
        self.file_object.seek(0, 0)
        text = self.file_object.read()
        text = text.split('\n')
        text.reverse()
        output = '\n'.join(text)
        self.file_object.close()

        with open(self.outfile, 'w') as f:
            f.write(output)

    def flush(self):
        self.file_object.close()
        self.file_object = open(self.filename, 'a+', encoding = 'utf-8')

Log().close()
log = Log()

In [ ]:
from datetime import datetime
print(datetime.now())

2021-02-18 09:17:11.854725


In [ ]:
cd /content/drive/MyDrive/Dissertation/

/content/drive/MyDrive/Dissertation


In [ ]:
class Opt:
    pass
opt = Opt()
opt.src_lang = 'es'
opt.trg_lang = 'en'
opt.num_mil = 1
opt.input_file = f'data/{opt.src_lang}/ParaCrawl.{opt.src_lang}-{opt.trg_lang}.'
opt.model_file = f'data/{opt.src_lang}/SPM-{opt.num_mil}m-8k.{opt.src_lang}-{opt.trg_lang}.'
opt.interim_file = f'data/{opt.src_lang}/ParaCrawl.{opt.src_lang}-{opt.trg_lang}.{opt.num_mil}m.'
opt.dataset = f'data/{opt.src_lang}/tokenized_dataset_{opt.src_lang}_{opt.num_mil}m'
opt.dev_dataset = f'data/{opt.src_lang}/DEV-{opt.src_lang}-{opt.trg_lang}.'
# opt.model_file = 'data/SPM-1m-8k.fr-en.'
opt.max_len = 150
opt.dev_dataset = f'data/{opt.src_lang}/DEV-{opt.src_lang}-{opt.trg_lang}.'


opt.src_data_path = opt.interim_file + opt.src_lang
opt.trg_data_path = opt.interim_file + opt.trg_lang
opt.vocab_size = 8000
opt.tokensize = 2048
opt.print_every = 200
opt.save_every = 5000
opt.epochs = 10
opt.warmup_steps = 16000
opt.keep_training = False

opt.path = f'{opt.src_lang}-en-models'
opt.model_prefix = f'{opt.src_lang}-en-model-'
# optim_file = 'data/optim_state_dict'
opt.model_dim = 512
opt.heads = 8
opt.N = 6
opt.args = (opt.vocab_size, opt.vocab_size, 
            opt.model_dim, opt.model_dim*4, 
            opt.heads, opt.N, opt.max_len, 0.1, True)
opt.log = log

In [ ]:
def move(opt):
    def move_lang(lang):
        inpFile = open(opt.input_file + lang, 'r', encoding='utf-8')
        intFile = open(opt.interim_file + lang, 'w', encoding='utf-8')

        for i in tnrange(int(opt.num_mil * 10**6)):
            intFile.write(inpFile.readline())
        inpFile.close()
        intFile.close()
    move_lang(opt.src_lang)
    move_lang(opt.trg_lang)
# move(opt)

In [ ]:
# ! spm_train --input=data/es/ParaCrawl.es-en.1m.es --model_prefix=data/es/SPM-1m-8k.es-en.es --vocab_size=8000 --character_coverage=1.0 --model_type=bpe --pad_id=3 --bos_id=-1 --eos_id=-1

In [ ]:
# ! spm_train --input=data/es/ParaCrawl.es-en.1m.en --model_prefix=data/es/SPM-1m-8k.es-en.en --vocab_size=8000 --character_coverage=1.0 --model_type=bpe --pad_id=3

In [ ]:
def create_fields(opt):
    
    print('creating fields')
    
    if not os.path.exists(f"{opt.model_file}{opt.src_lang}.model"):
        print('building src processor') 
        raise Exception

    
    if not os.path.exists(f"{opt.model_file}{opt.trg_lang}.model"):
        print('building trg processor')
        raise Exception

    print("initlizing sentence processors")
    opt.src_processor = spm.SentencePieceProcessor()
    opt.src_processor.Init(model_file=f'{opt.model_file}{opt.src_lang}.model')
    opt.trg_processor = spm.SentencePieceProcessor()
    opt.trg_processor.Init(model_file=f'{opt.model_file}{opt.trg_lang}.model')

    opt.src_pad = opt.src_processor.pad_id()
    opt.trg_pad = opt.trg_processor.pad_id()
    opt.trg_bos = opt.trg_processor.bos_id()
    opt.trg_eos = opt.trg_processor.eos_id()


def create_dataset(opt):

    opt.bins = [i for i in range(10, opt.max_len+1)]

    if opt.dataset is not None and os.path.exists(opt.dataset):
        print('loading saved dataset...')
        with open(opt.dataset, 'rb') as f:
            opt.src_bins = pickle.load(f)
            opt.trg_bins = pickle.load(f)

        print({s: len(opt.src_bins[s]) for s in opt.bins})
        return

    print('reading datasets')
    with open(opt.src_data_path, 'r', encoding='utf-8') as f:
        opt.src_data = f.read().split('\n')
    with open(opt.trg_data_path, 'r', encoding='utf-8') as f:
        opt.trg_data = f.read().split('\n')

    opt.src_bins = {i:[] for i in opt.bins}
    opt.trg_bins = {i:[] for i in opt.bins}

    print('tokenizing and bining...')
    for i in tnrange(len(opt.src_data)):
        src = opt.src_data[i]
        trg = opt.trg_data[i]
    # for i, (src, trg) in enumerate(zip(opt.src_data, opt.trg_data)):
        src = opt.src_processor.encode(src)
        trg = [opt.trg_bos] + opt.trg_processor.encode(trg) + [opt.trg_eos]
        opt.src_data[i] = 0
        opt.trg_data[i] = 0

        lsrc= len(src)
        ltrg = len(trg)
        if lsrc > opt.max_len or ltrg > opt.max_len:
            continue
        
        for v in opt.bins:
            if lsrc <= v and ltrg <= v:
                for i in range(lsrc, v):
                    src.append(opt.src_pad)
                for i in range(ltrg, v):
                    trg.append(opt.trg_pad)
                
                opt.src_bins[v].append(src)
                opt.trg_bins[v].append(trg)
                break

    if opt.dataset is not None:
        with open(opt.dataset, 'wb') as f:
            pickle.dump(opt.src_bins, f)
            pickle.dump(opt.trg_bins, f)

    print({s: len(opt.src_bins[s]) for s in opt.bins})

create_fields(opt)
create_dataset(opt)


creating fields
initlizing sentence processors
loading saved dataset...
{10: 36, 11: 9, 12: 20, 13: 38, 14: 28, 15: 43, 16: 25, 17: 27, 18: 22, 19: 15, 20: 6, 21: 8, 22: 9, 23: 11, 24: 23, 25: 23, 26: 291, 27: 676, 28: 636, 29: 682, 30: 1408, 31: 2392, 32: 3731, 33: 7784, 34: 8105, 35: 11320, 36: 11992, 37: 13262, 38: 15144, 39: 16658, 40: 18891, 41: 19362, 42: 21538, 43: 21926, 44: 24012, 45: 24913, 46: 24096, 47: 24438, 48: 24528, 49: 24098, 50: 23286, 51: 21698, 52: 20120, 53: 17725, 54: 16078, 55: 14397, 56: 12427, 57: 11141, 58: 9615, 59: 8435, 60: 7671, 61: 6594, 62: 6348, 63: 5375, 64: 4922, 65: 4320, 66: 3943, 67: 3725, 68: 3454, 69: 3245, 70: 3279, 71: 3244, 72: 3174, 73: 3802, 74: 3842, 75: 3293, 76: 3382, 77: 3573, 78: 3974, 79: 3992, 80: 4210, 81: 4381, 82: 4616, 83: 4916, 84: 6176, 85: 5964, 86: 5940, 87: 5527, 88: 5678, 89: 5716, 90: 5824, 91: 6204, 92: 7026, 93: 6693, 94: 6590, 95: 6980, 96: 7998, 97: 7366, 98: 7461, 99: 7594, 100: 8313, 101: 8022, 102: 7863, 103: 7985, 

In [ ]:
opt.dev_dataset = f'data/{opt.src_lang}/DEV-{opt.src_lang}-{opt.trg_lang}.'

with open(opt.dev_dataset + opt.src_lang, 'r', encoding='utf-8') as f:
    opt.dev_src_sentences = f.read().split('\n')[:2000]
with open(opt.dev_dataset + opt.trg_lang, 'r', encoding='utf-8') as f:
    opt.dev_trg_sentences = f.read().split('\n')[:2000]

In [ ]:
opt.train_len = sum((len(opt.src_bins[v])*v) // opt.tokensize + 1 for v in opt.bins) 
print(opt.train_len)
print(sum(len(opt.src_bins[v]) for v in opt.bins))

35390
966398


In [ ]:
import numpy as np

class Save:
    def __init__(self, model_state_dict=None, optim_state_dict=None):
        self.model_state_dict = model_state_dict
        self.optim_state_dict = optim_state_dict

    def save(self, filename):
        torch.save(self.model_state_dict, f'{filename}.model')
        torch.save(self.optim_state_dict, f'{filename}.optim')
    
    def load(self, filename):
        
        if torch.cuda.is_available():
            device = torch.device('cuda:0')
        else:
            device = torch.device('cpu')

        self.model_state_dict = torch.load(f'{filename}.model', map_location=device)
        self.optim_state_dict = torch.load(f'{filename}.optim', map_location=device)

In [ ]:
def batch(opt):
    max_count = {v:(len(opt.src_bins[v])*v) // opt.tokensize for v in opt.bins}
    # print(max_count)
    cur_count = {v:0 for v in opt.bins}
    batch_sizes = {v: opt.tokensize // v for v in opt.bins}

    step = 0
    while (len(max_count)):
        v = np.random.choice(list(max_count.keys()))
        i = cur_count[v]
        cur_count[v] += 1
        j = cur_count[v]

        step += 1
        if step < opt.starting_index: continue

        size = batch_sizes[v]
        src_list = opt.src_bins[v][i*size: j*size]
        trg_list = opt.trg_bins[v][i*size: j*size]

        if j > max_count[v]:
            if opt.keep_training: cur_count[v] = 0
            else: del max_count[v]

        if len(src_list) == 0:
            continue

        yield src_list, trg_list

In [ ]:
def nopeak_mask(size, opt):
    np_mask = np.triu(np.ones((1, size, size)),
    k=1).astype('uint8')
    np_mask =  Variable(torch.from_numpy(np_mask) == 0).to(device)
    return np_mask

def create_masks(src, trg, opt):
    
    src_mask = (src != opt.src_pad).unsqueeze(-2)

    if trg is not None:
        trg_mask = (trg != opt.trg_pad).unsqueeze(-2)
        size = trg.size(1) # get seq_len for matrix
        np_mask = nopeak_mask(size, opt).to(device)
        trg_mask = trg_mask & np_mask
        
    else:
        trg_mask = None
    return src_mask, trg_mask

def init_vars(src, model, opt):
    
    init_tok = opt.trg_bos
    src_mask = (src != opt.src_pad).unsqueeze(-2)
    e_output = model.encoder(src, src_mask)
    
    outputs = torch.LongTensor([[init_tok]]).to(opt.device)
    
    trg_mask = nopeak_mask(1, opt)
    
    out = model.linear(model.decoder(outputs, e_output, src_mask, trg_mask))
    out = F.softmax(out, dim=-1)
    
    probs, ix = out[:, -1].data.topk(opt.k)
    log_scores = torch.Tensor([math.log(prob) for prob in probs.data[0]]).unsqueeze(0)
    
    outputs = torch.zeros(opt.k, opt.max_len).long().to(opt.device)
    outputs[:, 0] = init_tok
    outputs[:, 1] = ix[0]
    
    e_outputs = torch.zeros(opt.k, e_output.size(-2),e_output.size(-1)).to(opt.device)
    e_outputs[:, :] = e_output[0]
    
    return outputs, e_outputs, log_scores

def k_best_outputs(outputs, out, log_scores, i, k):
    
    probs, ix = out[:, -1].data.topk(k)
    log_probs = torch.Tensor([math.log(p) for p in probs.data.view(-1)]).view(k, -1) + log_scores.transpose(0,1)
    k_probs, k_ix = log_probs.view(-1).topk(k)
    
    row = k_ix // k
    col = k_ix % k

    outputs[:, :i] = outputs[row, :i]
    outputs[:, i] = ix[row, col]

    log_scores = k_probs.unsqueeze(0)
    
    return outputs, log_scores

def beam_search(src, model, opt):
    

    outputs, e_outputs, log_scores = init_vars(src, model, opt)
    eos_tok = opt.trg_eos
    src_mask = (src != opt.src_pad).unsqueeze(-2)
    ind = None
    for i in range(2, opt.max_len):
    
        trg_mask = nopeak_mask(i, opt)

        out = model.linear(model.decoder(outputs[:,:i],
        e_outputs, src_mask, trg_mask))

        out = F.softmax(out, dim=-1)
    
        outputs, log_scores = k_best_outputs(outputs, out, log_scores, i, opt.k)
        
        ones = torch.nonzero(outputs == eos_tok)
        # ones = (outputs==eos_tok).nonzero() # Occurrences of end symbols for all input sentences.
        sentence_lengths = torch.zeros(len(outputs), dtype=torch.long).cuda()
        for vec in ones:
            i = vec[0]
            if sentence_lengths[i]==0: # First end symbol has not been found yet
                sentence_lengths[i] = vec[1] # Position of first end symbol

        num_finished_sentences = len([s for s in sentence_lengths if s > 0])

        if num_finished_sentences == opt.k:
            alpha = 0.7
            div = 1/(sentence_lengths.type_as(log_scores)**alpha)
            _, ind = torch.max(log_scores * div, 1)
            ind = ind.data[0]
            break
    
    if ind is None:
        # print(outputs)
        length = (outputs[0]==eos_tok).nonzero()[0]
        sentence_list = (outputs[0][1:length]).tolist()
        return ''.join(opt.trg_processor.decode(sentence_list)).replace('_', " ")
    
    else:
        length = (outputs[ind]==eos_tok).nonzero()[0]
        sentence_list = (outputs[0][1:length]).tolist()
        return ''.join(opt.trg_processor.decode(sentence_list)).replace('_', " ")

import re
def multiple_replace(dict, text):
  # Create a regular expression  from the dictionary keys
  regex = re.compile("(%s)" % "|".join(map(re.escape, dict.keys())))

  # For each match, look-up corresponding value in dictionary
  return regex.sub(lambda mo: dict[mo.string[mo.start():mo.end()]], text) 

def translate_sentence(sentence, model, opt):
    
    model.eval()
    sentence = Variable(torch.LongTensor([opt.src_processor.encode(sentence)])).to(opt.device)
    
    sentence = beam_search(sentence, model, opt)

    return  multiple_replace({' ?' : '?',' !':'!',' .':'.','\' ':'\'',' ,':','}, sentence)

In [ ]:
if torch.cuda.is_available():
    dev = "cuda:0"
else:
    dev = "cpu"

device = torch.device(dev)
opt.device = device

In [ ]:
model = Transformer(*args)
optim = torch.optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)
opt.save_model = Save()
starting_index = 0
for p in model.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

# initializing the parameters of the model.

if not os.path.exists(path):
    if not os.path.exists(path):
        os.mkdir(path)
    log.print(f"No {path} found. Created a new path directory and started using xavier_uniform")
else:
    for i in os.walk(path):
        break
    i = i[2]
    m = 0
    mf = None
    suffix_len = len('.model')
    for file in i:
        if model_prefix not in file or '.model' not in file: continue
        
        num = int(file[len(model_prefix):-suffix_len])
        if num > m:
            m = num
            mf = file[:-suffix_len]
                
    if mf is not None:
        log.print(f"Starting from last saved {mf}")
        opt.save_model.load(f'{path}/{mf}')
        model.load_state_dict(opt.save_model.model_state_dict)
        optim.load_state_dict(opt.save_model.optim_state_dict)
        starting_index = m
    else:
        log.print(f"Starting from xavier_uniform distribution")
opt.starting_index = starting_index

Starting from last saved es-en-model-120000


In [ ]:
import math
def evaluate(model, opt):
    model.eval()

    # translated = []
    start = time.time()

    refs = []
    hyp = []
    opt.skip = []

    # print(len(opt.dev_src_sentences))
    tk2 = tnrange(len(opt.dev_src_sentences))
    for i in tk2:
        sentence = opt.dev_src_sentences[i]
        try:
            translated = translate_sentence(sentence.lower(), model, opt)
        except Exception as e:
            opt.skip.append(i)
            # raise e
        else:
            refs.append(opt.dev_trg_sentences[i])
            hyp.append(translated)
            if i < 10:
                hyp_print = f"hyp: {hyp[-1]}"
                ref_print = f"ref: {refs[-1]}"
                log.print(hyp_print, shell=False)
                log.print(ref_print, shell=False)
                log.print('', shell=False)
                log.flush()
    # print()
    
        if i == len(opt.dev_src_sentences) - 1:
            opt.hyp = hyp
            opt.refs = refs

            bleu = sacrebleu.corpus_bleu(opt.hyp[:-1], [opt.refs[:-1]])
            ter = sacrebleu.corpus_ter(opt.hyp[:-1], [opt.refs[:-1]])
            tk2.set_postfix_str(f"({ter} || {bleu})")
            log.print(f'{ter} || {bleu}', shell=False)
    return bleu, ter

# opt.k = 10
# evaluate(model, opt)

In [ ]:
log.flush()
k = 10
# to adjust for models already present
opt.device = device
def train_model(model, opt):
    model.train()
 
    # starting_index += _starting_index
 
    start = time.time()
    
    total_loss = 0
    total_loss_at_save = 0
    last_loss = 0
    r = opt.save_every / opt.print_every
 
    prev_scores = []
    tk0 = tnrange(opt.epochs)
    for epoch in tk0:
        temp = time.time()
        tk1 = tnrange(opt.train_len)
        batch_gen = batch(opt)
        last_batch_loss = 0
        for i in tk1:
            src_lis, trg_lis = next(batch_gen)
            optim.zero_grad()
            try:
                src_tensor = torch.LongTensor(src_lis).to(opt.device)
                src_tensor.requires_grad = False
                trg_np = np.array(trg_lis)
                trg_tensor = torch.LongTensor(trg_np[:, :-1]).to(device)
                trg_tensor.requires_grad = False
            except Exception as e:
                del src_tensor
                continue
            src_mask, trg_mask = create_masks(src_tensor, trg_tensor, opt)
 
            preds = model(src_tensor, trg_tensor, src_mask, trg_mask)
            target = torch.LongTensor(trg_np[:, 1:]).to(device).contiguous().view(-1)
            preds = preds.view(-1, preds.size(-1))
            loss = F.cross_entropy(preds, target, ignore_index=opt.trg_pad)
 
            loss.backward()
            total_loss += loss.item()
            optim.step()
            step = starting_index + int(epoch * opt.train_len) + i + 1
            optim.param_groups[0]['lr'] = (model_dim ** (-0.5)) * min(step ** (-0.5), step * (opt.warmup_steps ** (-1.5)))
 
            del src_mask, src_tensor, trg_mask, trg_tensor, preds, loss
            torch.cuda.empty_cache()
 
            if (step) % opt.print_every == 0:
                diff = total_loss - last_batch_loss
                diff = '%.3f'%diff
                last_batch_loss = total_loss

                avg = "%.3f" % (total_loss)
                t = "%.3f" % (time.time() - temp)
                tt = time.strftime('%H:%M:%S', time.gmtime(time.time() - start))

                output = f"time: {t}s, total: {tt}, loss = {avg}, step = {step}, diff = {diff}"
                log.print(output, shell=False)
                tk1.set_postfix_str(", " + output + '\n')
 
                total_loss_at_save += total_loss
                total_loss = 0
                temp = time.time()
 
            if (step) % opt.save_every == 0:
                model_name = f'{path}/{model_prefix}{step}'
                opt.save_model.model_state_dict = model.state_dict()
                opt.save_model.optim_state_dict = optim.state_dict()
                opt.save_model.save(model_name)
 
                avg = (total_loss_at_save / r)
                total_loss_at_save = 0
                diff = avg - last_loss
                last_loss = avg
 
                avg = '%.3f' % avg
                diff = '%.3f' % diff
                output = f"Saving model: {model_name} | avg_loss: {avg} | diff: {diff}"
                log.print(output, shell=False)
                tk0.set_postfix_str(',' + output + '\n')            
 
            if step % opt.train_len == 0:
                bleu_score, ter = evaluate(model, opt)
                score = '%.3f' % bleu_score.score
 
                prev_scores.append(bleu_score.score)
                if len(prev_scores) > 5: prev_scores.pop(0)
 
                prev_avg_score = sum(prev_scores) / len(prev_scores)
 
                # if len(prev_scores) == 5 and prev_avg_score - bleu_score.score < epsilon:
                #     break
                    
                log.print(f"{ter} || {bleu_score}")
                temp = time.time()
                model.train()
 
        log.flush()
 
 
if __name__ == '__main__':
 
    try:
        # print(eval())
        opt.k = 10
        opt.print_every = 200 
        opt.save_every = 5000
        opt.epochs = 10
        opt.warmup_steps = 16000
        opt.keep_training = True
        train_model(model, opt)
 
    except Exception as e:
        log.print(e, type=Log.ERROR)
        log.flush()
        raise e